In [3]:
import requests
# from pyden.connect.gsearch_api.google import google
# from pyden.newspaper.newspaper import Article
from newspaper import Article
# from pyden.google_api.search import gsearch

import time
import unidecode


In [4]:
toscrape = ["https://amwalalghad.com", "https://www.mubasher.info/markets/ADX/stocks/NBAD/news", "https://amwalalghad.com", "https://www.alarabiya.net/ar/tools"]


In [5]:
# define using se scraper and parsing with newspaper 3k
class Object(object):
    pass
def gsearch(term, pages = 1):
    term = term.replace("\"","").replace("\'","").replace("-", "")
    term += " -filetype:pdf"  #TODO: be able to read pdfs
    data = '{ "browser_config": { "random_user_agent": true },"scrape_config": { "search_engine": "google","keywords": ["'+term+'"],"num_pages": '+str(pages)+'}}'
    header = {'Content-Type': 'application/json'}
    results = requests.post("http://0.0.0.0:3000", data = data, headers = header).json()
    output = []

    for num in range(int(pages)):
        if str(num+1) in results['results'][term]:
            output.extend([i for i in results['results'][term][str(num+1)]['results']])
    return output

def search(query, link = ""):
    unaccented_string = unidecode.unidecode(query)
    query_outs = gsearch(unaccented_string)
    titles = []
    labels = []
    para = []
    rm = []
    outputs = [Object() for _ in query_outs]

    for i, qo in enumerate(query_outs):
        query = outputs[i]; query.link = qo['link']
        # if not query.link or query.link.replace('/', u'\u2215') == link[0]:  # the links have '/' replaced
        #     rm.append(i)
        #     continue
        art = Article(query.link)  # if you are an article, parse and
        try:
            start = time.time()
            art.download()
            if not art.html:
                rm.append(i)
                continue
            print(query.link)
            print("took " + str(time.time() - start))
            art.parse()  # some articles cannot be parsed
            para.append(art.text.split("\n\n"))  # this makes paras a list of lists
            toadd = query.link.replace('/', u'\u2215')
            if len(query.link) > 255:
                toadd = toadd[:255]
            labels.append(toadd)  # titles are the labels that things are saved by
            query.title = art.title
            if art.title == "":
                query.title = qo[i]['title']  # this is just going to be what the title was on google
            query.sources = art.source_url
            query.authors = art.authors
            if art.publish_date:
                query.date = art.publish_date.strftime('%m/%d/%Y')
        except Exception as e:
            print(e)  # if u cant load it uhhhh just dont do anything
    for index in sorted(rm, reverse=True):
        del outputs[index]
    return outputs, labels, para, titles



In [9]:
results = search("site:" + toscrape[0] + " fab")

text/html; charset=UTF-8


text/html; charset=UTF-8
text/html; charset=UTF-8


text/html; charset=UTF-8


text/html; charset=UTF-8


text/html; charset=UTF-8


text/html; charset=UTF-8


text/html; charset=UTF-8


text/html; charset=UTF-8


In [10]:
print(results[2]) 

In [75]:
model_path = "finBERT/models/trc2/"

In [68]:
# path to trc2 pytorch model # for some reason flask app doesn't work
paragraph = results[2][1][1]
# sample = '{"text": ' + paragraph + '}'
sample = {"text": paragraph}
header = {'Content-Type': 'application/json'}
def get_sentiment(text):
    results = requests.post("http://0.0.0.0:8080", json = sample, headers = header)
print(type(get_sentiment(sample)))

In [1]:
# paste in flask app prediction code here, but remove the flask
import sys
sys.path.insert(0, "/finBERT")
from finbert.finbert import predict
from pytorch_pretrained_bert.modeling import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(model_path,num_labels=3,cache_dir=None)


ModuleNotFoundError: No module named 'finbert'

In [2]:
# take a list of sentiment analysis of all the sentences that have the stock name
sentiments = []
for article in results[2]:
    per_article = []
    for paragraph in article:
        if paragraph:
            per_article.append(predict(paragraph,model))
    sentiments.append(per_article)

NameError: name 'results' is not defined

In [87]:
sentiments[0][0]

,sentence,logit,prediction,sentiment_score
0,Egypt has tapped Emirates NBD and First Abu Dh...,"[0.29286325, 0.2198136, 0.48732308]",neutral,0.07305


In [86]:
results[0][0].date

'06/17/2020'